In [0]:
from pyspark.sql.functions import *
from pyspark.sql import *

Reading a Line Delimetered Json (Each Line Will have complete 1 record ONLY)

In [0]:
df = spark.read.format("json")\
    .option("header","true")\
    .option("mode","PERMISSIVE")\
    .load("/Volumes/workspace/default/rajatlearningdata/PySpark_data/line_delimited_json.json")

df.dropna().show()

Lets read a Json File in Which 1 Record have one Extra column as Gender (for all other record with no  column as Gender it will set NULL)

In [0]:
df2 = spark.read.format("json")\
  .option("header","true")\
  .option("mode","PERMISSIVE")\
  .load("/Volumes/workspace/default/rajatlearningdata/PySpark_data/single_file_json_with_extra_fields.json")

df2.show()

**.option("multiLine","true") : Lets Try To Read Multi-Line Json **
- While Reading a multiline json we have to read it as a list, otherwise it will show only 1st element

Lets First Try to Read Multi-Line Incorrect JSON (not as a list -OR- not as a list of dictionaries)

In [0]:
df3 = spark.read.format("json")\
  .option("header","true")\
  .option("multiLine","true")\
  .option("mode","permissive")\
  .load("/Volumes/workspace/default/rajatlearningdata/PySpark_data/multi_line_incorrect_json.json")

df3.show()

%md
Now lets Try to Read Multi-Line correct JSON (as a list -OR- a list of dictionaries)

In [0]:
df3 = spark.read.format("json")\
  .option("header","true")\
  .option("multiLine","true")\
  .option("mode","permissive")\
  .load("/Volumes/workspace/default/rajatlearningdata/PySpark_data/multi_line_correct_json.json")

df3.show()

### Handling Corrupted JSON : automatic
- It will automatic create "_corrupt_record" Column and put corrupt record inside that

In [0]:
df4 = spark.read.format("json")\
  .option("header","true")\
  .option("mode","permissive")\
  .load("/Volumes/workspace/default/rajatlearningdata/PySpark_data/corrupted_json.json")
  
df4.show()

### Now Lets Handle /  FLATTEN - NESTED JSON

Lets first try to Read Nested - JSON Data as a simple Multiline JSON

In [0]:
restaurant_json_data = spark.read.format("json")\
  .option("header","true")\
  .option("inferSchema","true")\
  .option("multiine","true")\
  .load("/Volumes/workspace/default/rajatlearningdata/PySpark_data/resturant_json_data.json")

resturant_json_data.show()

In [0]:
restaurant_json_data.printSchema()

Above We can see in Nested JSON, some elements are of "array" type (like : restaurant), we will "explode()" them  
Some Elements are of "struct" type (like : R) , we can access them using '.' (dot) operator

In [0]:
restaurant_json_data.select("*",explode("restaurants").alias("new_restaurant")).drop("restaurants").printSchema()

- Above we can see now column "new_restaurant" is of struct type , we can access it's element using '.' (dot)
- try to fetch re_id

In [0]:
restaurant_json_data.select("*",explode("restaurants").alias("new_restaurant")).drop("restaurants").select("new_restaurant.restaurant.R.res_id").show(5)

While using Explode if any Column have null then explode remove that entire column . "explode()" won't give us result. We have to Use " explode_outer() "

In [0]:
restaurant_json_data.select("*",explode("restaurants").alias("new_restaurant")).drop("restaurants")\
  .select("*","new_restaurant.restaurant.R.res_id",
          explode("new_restaurant.restaurant.establishment_types").alias("establishment_types_new")).drop("new_restaurant").show()

Now Lets use explode_outer()

In [0]:
restaurant_json_data.select("*",explode("restaurants").alias("new_restaurant")).drop("restaurants")\
  .select("*","new_restaurant.restaurant.R.res_id",
          explode_outer("new_restaurant.restaurant.establishment_types").alias("establishment_types_new")).drop("new_restaurant").show()